# SKA-Mid conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

XRADIO version 1.1.12a0 already installed.


## Download dataset

In [2]:
import toolviper

toolviper.utils.data.download(file="AA2-Mid-sim_00000.ms")

[2026-01-21 13:14:42,730]     INFO    viperlog:  Downloading from [cloudflare] .... 


 Download List         
 ────────────────────── 
  AA2-Mid-sim_00000.ms

[2026-01-21 13:14:42,733]     INFO    viperlog:  File exists: AA2-Mid-sim_00000.ms 


Output()

## Convert to Processing Set

In [3]:
from xradio.measurement_set import convert_msv2_to_processing_set

ms_file = "AA2-Mid-sim_00000.ms"

main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "AA2-Mid-sim_00000.ps.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel_mode="none",
    persistence_mode='w',
    main_chunksize=main_chunksize,
)

[2026-01-21 13:14:43,877]     INFO    viperlog:  Updated partition scheme used: ['DATA_DESC_ID', 'OBSERVATION_ID'] 
[2026-01-21 13:14:43,879]     INFO    viperlog:  Number of partitions: 1 
[2026-01-21 13:14:43,879]     INFO    viperlog:  OBSERVATION_ID [0], DDI [0], STATE [None], FIELD [0], SCAN [0], EPHEMERIS [None] 
[2026-01-21 13:14:44,091]  WARNING    viperlog:  SOURCE table empty for (unique) source_id [0] and spectral_window_id 0. 


## Processing Set

In [4]:
ps_name = outfile

from xradio.measurement_set import open_processing_set

ps_xdt = open_processing_set(ps_store=outfile)
ps_xdt.xr_ps.summary()

,name,scan_intents,shape,execution_block_UID,polarization,scan_name,spw_name,spw_intents,field_name,source_name,line_name,field_coords,session_reference_UID,scheduling_block_UID,project_UID,start_frequency,end_frequency
0,AA2-Mid-sim_00000_0,[None],"(95, 1891, 3, 4)",---,"[XX, XY, YX, YY]",[0],spw_0,[UNSPECIFIED],[_0],[Unknown],[],"[fk5, 0h00m00.00s, -45d00m00.00s]",---,---,,950000000.0,9.500989e+08


In [5]:
ms_xdt = ps_xdt["AA2-Mid-sim_00000_0"]

In [6]:
ms_xdt.xr_ms.get_field_and_source_xds()

<xarray.DatasetView> Size: 16kB
Dimensions:                       (field_name: 1, sky_dir_label: 2,
                                   baseline_id: 1891, frequency: 3,
                                   polarization: 4, time: 95, uvw_label: 3)
Coordinates:
  * field_name                    (field_name) <U23 92B '_0'
  * sky_dir_label                 (sky_dir_label) <U3 24B 'ra' 'dec'
  * baseline_id                   (baseline_id) int64 15kB 0 1 2 ... 1889 1890
  * frequency                     (frequency) float64 24B 9.5e+08 ... 9.501e+08
  * polarization                  (polarization) <U2 32B 'XX' 'XY' 'YX' 'YY'
  * time                          (time) float64 760B 9.467e+08 ... 9.467e+08
  * uvw_label                     (uvw_label) <U1 12B 'u' 'v' 'w'
    source_name                   (field_name) <U7 28B dask.array<chunksize=(1,), meta=np.ndarray>
Data variables:
    FIELD_PHASE_CENTER_DIRECTION  (field_name, sky_dir_label) float64 16B dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes:
    type:     field_and_source

In [7]:
ms_xdt.antenna_xds

<xarray.DataTree 'antenna_xds'>
Group: /AA2-Mid-sim_00000_0/antenna_xds
    Dimensions:                 (time: 95, baseline_id: 1891, frequency: 3,
                                 polarization: 4, uvw_label: 3, antenna_name: 62,
                                 cartesian_pos_label: 3, receptor_label: 2)
    Coordinates:
      * antenna_name            (antenna_name) <U5 1kB 's0000' 's0001' ... 's0061'
      * cartesian_pos_label     (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
      * receptor_label          (receptor_label) <U5 40B 'pol_0' 'pol_1'
        mount                   (antenna_name) <U5 1kB dask.array<chunksize=(62,), meta=np.ndarray>
        polarization_type       (antenna_name, receptor_label) <U1 496B dask.array<chunksize=(62, 2), meta=np.ndarray>
        station_name            (antenna_name) <U5 1kB dask.array<chunksize=(62,), meta=np.ndarray>
        telescope_name          (antenna_name) <U35 9kB dask.array<chunksize=(62,), meta=np.ndarray>
    Inherited coordinates:
      * baseline_id             (baseline_id) int64 15kB 0 1 2 3 ... 1888 1889 1890
      * frequency               (frequency) float64 24B 9.5e+08 9.5e+08 9.501e+08
      * polarization            (polarization) <U2 32B 'XX' 'XY' 'YX' 'YY'
      * time                    (time) float64 760B 9.467e+08 ... 9.467e+08
      * uvw_label               (uvw_label) <U1 12B 'u' 'v' 'w'
    Data variables:
        ANTENNA_DISH_DIAMETER   (antenna_name) float64 496B dask.array<chunksize=(62,), meta=np.ndarray>
        ANTENNA_POSITION        (antenna_name, cartesian_pos_label) float64 1kB dask.array<chunksize=(62, 3), meta=np.ndarray>
        ANTENNA_RECEPTOR_ANGLE  (antenna_name, receptor_label) float64 992B dask.array<chunksize=(62, 2), meta=np.ndarray>
    Attributes:
        overall_telescope_name:  OSKAR 2.8.4-dev 2023-08-22 d1b86f8c
        relocatable_antennas:    False
        type:                    antenna

In [8]:
ms_xdt.ds

<xarray.DatasetView> Size: 35MB
Dimensions:                     (time: 95, baseline_id: 1891, frequency: 3,
                                 polarization: 4, uvw_label: 3)
Coordinates:
  * time                        (time) float64 760B 9.467e+08 ... 9.467e+08
  * baseline_id                 (baseline_id) int64 15kB 0 1 2 ... 1889 1890
  * frequency                   (frequency) float64 24B 9.5e+08 ... 9.501e+08
  * polarization                (polarization) <U2 32B 'XX' 'XY' 'YX' 'YY'
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
    baseline_antenna1_name      (baseline_id) <U5 38kB dask.array<chunksize=(1891,), meta=np.ndarray>
    baseline_antenna2_name      (baseline_id) <U5 38kB dask.array<chunksize=(1891,), meta=np.ndarray>
    field_name                  (time) <U23 9kB dask.array<chunksize=(95,), meta=np.ndarray>
    scan_name                   (time) <U21 8kB dask.array<chunksize=(95,), meta=np.ndarray>
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 1MB dask.array<chunksize=(20, 1891), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 2MB dask.array<chunksize=(20, 1891, 1, 4), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 1MB dask.array<chunksize=(20, 1891), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 4MB dask.array<chunksize=(20, 1891, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 17MB dask.array<chunksize=(20, 1891, 1, 4), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 9MB dask.array<chunksize=(20, 1891, 1, 4), meta=np.ndarray>
Attributes:
    creation_date:     2026-01-21T18:14:43.923847+00:00
    creator:           {'software_name': 'xradio', 'version': '1.1.12a0'}
    data_groups:       {'base': {'correlated_data': 'VISIBILITY', 'date': '20...
    observation_info:  {'observer': ['fred'], 'observing_log': "['']", 'proje...
    processor_info:    {'sub_type': '', 'type': ''}
    schema_version:    4.0.0
    type:              visibility